# Traitement des données STOC-EPS Auvergne

In [ ]:
import sys
sys.version

In [ ]:
import re
import datetime as dt
import pandas as pd
import numpy as np

## Fichier Romain STOC/SHOC juillet 2017

### 0. Paramètres

In [ ]:
annees = [2017]

### 1. Chargement des données, correction des noms de lieux-dits et simplification du jeu de colonnes

In [ ]:
strAnnees = str(annees[0])
if len(annees) > 1:
    strAnnees += '-' + str(annees[-1])
strAnnees

In [ ]:
dfStocBrut = pd.DataFrame()
for annee in annees:
    dfStocBrutAnnee = pd.read_excel('Export STOC-SHOC Auvergne {}.xlsx'.format(annee))
    dfStocBrut = dfStocBrut.append(dfStocBrutAnnee)

len(dfStocBrut)

In [ ]:
KLieuDitDeviant = re.compile(r'([0-9]+) \(point ([0-9]+)\)')
def corrigerLieuDit(lieuDit):
    
    mo = KLieuDitDeviant.match(lieuDit)
    if mo:
        print(lieuDit, mo.group(1) + '_' + mo.group(2))
        lieuDit = mo.group(1) + '_' + mo.group(2)
        
    return lieuDit

dfStocBrut['Lieu-dit'] = dfStocBrut['Lieu-dit'].apply(corrigerLieuDit)

In [ ]:
dfStocBrut.columns

In [ ]:
# On ne s'intéresse qu'au STOC EPS
dfStocBrut = dfStocBrut[dfStocBrut['Protocole'] == 'STOC_EPS']
len(dfStocBrut)

In [ ]:
# Suppression des colonnes inutiles, fusion des colonnes similaires, ...
dfStoc = dfStocBrut[['Ref', 'Nom latin', 'Ordre systématique', 
                     'Date', 'Année', 'Heure début', 'Lieu-dit', 'Commune', 'Département', 'Altitude',
                     'Nombre', 'Détails',
                     'Prénom', 'Nom', 'Remarque', 'Remarque privée', 'Commentaire']].copy()
#del dfStocBrut

In [ ]:
dfStoc.set_index('Ref', inplace=True)

In [ ]:
# Fusion des colonnes de commentaires
dfStoc['Remarque'] = dfStoc['Remarque'].fillna(value='')
dfStoc['Remarque privée'] = dfStoc['Remarque privée'].fillna(value='')
dfStoc['Commentaire'] = dfStoc['Commentaire'].fillna(value='')

def mergeComments(sRow):
    strOut = sRow['Remarque']
    if sRow['Remarque privée']:
        if strOut:
            strOut += ' ; '
        strOut += sRow['Remarque privée']
    if sRow['Commentaire']:
        if strOut:
            strOut += ' ; '
        strOut += sRow['Commentaire']
    return strOut

dfStoc.Remarque = dfStoc.apply(mergeComments, axis=1)
dfStoc.drop(['Remarque privée', 'Commentaire'], axis=1, inplace=True)

In [ ]:
# Fusion des colonnes observateur
dfStoc['Observateur'] = dfStoc.apply(lambda sRow: sRow['Nom'] + ' ' + sRow['Prénom'], axis=1)
dfStoc.drop(['Nom', 'Prénom'], axis=1, inplace=True)

In [ ]:
dfStoc = dfStoc.reindex(columns=['Nom latin', 'Date', 'Lieu-dit', 'Nombre', 'Détails',
                                 'Observateur', 'Heure début', 'Commune', 'Département', 'Altitude',
                                 'Remarque', 'Ordre systématique', 'Année'])

In [ ]:
# Nb total de données brutes
len(dfStoc)

### 2. Sélection éventuelles des données à traiter.

In [ ]:
#dfStoc = dfStoc[dfStoc['Nom latin'] == 'Turdus merula']

In [ ]:
#dfStoc = dfStoc[dfStoc['Observateur'] == 'Meuret Jean-Philippe']

### 3. Quelques chiffres bruts

In [ ]:
# Espèces contactées.
dfStoc['Nom latin'].unique(), len(dfStoc['Nom latin'].unique())

In [ ]:
# Nb total de données, d'individus
len(dfStoc), dfStoc.Nombre.sum()

In [ ]:
# Nbre de sites inventoriés.
len(dfStoc['Lieu-dit'].unique()), dfStoc['Lieu-dit'].unique()

In [ ]:
# Nbre d'inventaires (STOC => normalement 2 par année, mais il y a des trous)
len(dfStoc.groupby(['Lieu-dit', 'Date']).first())

In [ ]:
# Petit avant-goût
dfStoc.head()

### 4. Nbre de données par classe de distance

In [ ]:
# Effectif par classe de distance { dist => eff. }, à partir de la colonne DETAIL,
# supposée formatée ainsi : "<n1>x nia nia ... (Distance moins de <d1>m) / ...
#                            ... / <np>x nia nia ... (Distance plus de <dp>m) / <np>x nia nia ... (En transit)"
KDistanceClassValues = { "25" : 25/2.0, "100" : (100+25)/2.0, "200" : (100+200)/2.0, "Plus" : 400.0 }
KReNumPerDistExpect = re.compile(r"([0-9]+)x(.*)\((.*) ([0-9]+)m\)")
KReNumPerDistTrans = re.compile(r"([0-9]+)x(.*)\(En transit\)")
KReNumPerDistOther = re.compile(r"([0-9]+)x.*")
categories = dict()
ignored = dict()
def details2Distances(strDetail):

    numPerDist = {}
    rawNumDists = strDetail.split("/")
    for rawNumDist in rawNumDists:
        
        rawNumDist = rawNumDist.replace('Distance plus de', '>')
        mo = KReNumPerDistExpect.match(rawNumDist.strip())
        if mo:
            num = int(mo.group(1))
            cat = mo.group(2).strip()
            categories[cat] = categories.get(cat, 0) + 1
            if 'poussin' in cat or '1ère année' in cat:
                print("'{}' dans '{}' trop jeune".format(rawNumDist, strDetail))
                ignored['jeune'] = ignored.get('jeune', 0) + num
                continue # On ignore les trop jeunes pour se reproduire sans faire d'histoires
                
            if mo.group(3) == "<":
                rawDist = mo.group(4)
            else:
                rawDist = "Plus"
            if rawDist not in KDistanceClassValues:
                ignored['bad-class'] = ignored.get('bad-class', 0) + num
                print("Attention, donnée ignorée : Classe de distance inconnue '{}' dans colonne DETAIL : '{}'" \
                      .format(rawDist, strDetail))
                print ("  On attendait : moins de 25, 100, 200 ou plus de 200")
                break

            numPerDist[KDistanceClassValues[rawDist]] = num
            
        elif "En transit" in rawNumDist:
            mo = KReNumPerDistTrans.match(rawNumDist.strip())
            num = 0 if not mo else int(mo.group(1))
            ignored['transit'] = ignored.get('transit', 0) + num
            if num == 0:
                print("Attention, donnée ignorée : Nb '{}' malformé dans colonne DETAIL : '{}'".format(rawNumDist, strDetail))
                ignored['bad-num'] = ignored.get('bad-num', 0) + 1
            continue # On ignore les "en vol" sans faire d'histoires
            
        else: # Autres cas : on essaie de compter quand même mais on râle
            mo = KReNumPerDistOther.match(rawNumDist.strip())
            num = 0 if not mo else int(mo.group(1))
            ignored['other'] = ignored.get('other', 0) + num
                
            print("Attention, donnée ignorée : Colonne DETAIL malformée : '{}' dans '{}'".format(rawNumDist, strDetail))
            print("  On attendait : <n1>x nia nia ... (< <d1>m) / ... "
                  "/ <np>x nia nia ... (> <dp>m) / <np>x nia nia ... (En transit)")
            if num == 0:
                print("Attention, donnée ignorée : Nb '{}' malformé dans colonne DETAIL malformée : '{}'".format(rawNumDist, strDetail))
                ignored['bad-num'] = ignored.get('bad-num', 0) + 1

    return pd.Series(numPerDist)

In [ ]:
distClassCols = ['Dist'+ str(d) for d in KDistanceClassValues.keys()]

dfStoc['Détails'].fillna(value='', inplace=True)
dfStoc[distClassCols] = dfStoc['Détails'].apply(details2Distances)

In [ ]:
# Catégories trouvées et nb d'individus
categories

In [ ]:
dfStoc['DistTot'] = dfStoc[['Dist25', 'Dist100', 'Dist200', 'DistPlus']].sum(axis=1).fillna(value=0).astype(int)

In [ ]:
dfStoc = dfStoc.reindex(columns=['Nom latin', 'Date', 'Lieu-dit', 'Dist25', 'Dist100', 'Dist200', 'DistPlus',
                                 'Observateur', 'Heure début', 'Commune', 'Département', 'Altitude',
                                 'Nombre', 'Détails',  'Remarque', 'Ordre systématique', 'Année', 'DistTot'])

In [ ]:
dfStoc.head()

In [ ]:
len(dfStoc)

In [ ]:
# Comparaison nb individu total et somme des classes de distances : différence normale =
# - les 'en transit' ignorés (TurMer : 36)
# - et qq autres, faute du saisisseur (TurMer : 4)
#   Ex: '1x' sans classe de distance
dfStoc['Nombre'].sum(), int(dfStoc['DistTot'].sum()), dfStoc['Nombre'].sum() - int(dfStoc['DistTot'].sum())

In [ ]:
# Majorité des explications : Nb individus ignorés, par classe de raison
ignored

In [ ]:
# Voici les lignes coupables des différences en questions
dfDistErr = \
  dfStoc[dfStoc['DistTot'] != dfStoc['Nombre']] \
        [['Nom latin', 'Lieu-dit', 'Dist25', 'Dist100', 'Dist200', 'DistPlus', 'DistTot',
          'Nombre', 'Détails', 'Date', 'Observateur']]
dfDistErr

### 5. Individualisation des données

In [ ]:
indivDataCols = ['Nom latin', 'Date', 'Lieu-dit', 'Distance', 'Observateur']
def generateIndivData(sGroupData):
    
    indivData = list()
    
    indivTmpl = pd.Series(data=[np.nan if col == 'Distance' else sGroupData[col] for col in indivDataCols],
                          index=indivDataCols)
    
    #print(sGroupData)
    
    for distClass, distValue in KDistanceClassValues.items():
        nbIndiv = sGroupData['Dist' + distClass]
        if nbIndiv > 0:
            for ind in range(int(nbIndiv)):
                indivDatum = indivTmpl.copy()
                indivDatum['Distance'] = distValue
                indivData.append(indivDatum)
            
    
    #print(indivData)
    
    return indivData

In [ ]:
dfStocIndiv = pd.DataFrame(columns=indivDataCols)
for ref, sGroupData in dfStoc.iterrows():
    dfStocIndiv = dfStocIndiv.append(pd.DataFrame(data=generateIndivData(sGroupData)), ignore_index=True)

In [ ]:
# Le nb d'individu total (comptés dans les classes de distance) n'a normalement pas changé lors l'individualisation
assert len(dfStocIndiv) == dfStoc['DistTot'].sum()

In [ ]:
# Mais comme certaines données ne sont pas classées en distances,
# on a peut-être perdu des sites / lieux-dit inventoriés
# En voici le nombre.
len(dfStoc['Lieu-dit'].unique()) - len(dfStocIndiv['Lieu-dit'].unique())

In [ ]:
# Voici les données avec 0 donnée classée en distance au moins 1 fois
dfNoDist = dfDistErr[dfDistErr.DistTot == 0]
dfNoDist

In [ ]:
# Nbre de lieux-dits associés
len(dfNoDist['Lieu-dit'].unique())

In [ ]:
# Mais certains de ces lieux ont d'autre données correctement classées en distance
len(set(dfStocIndiv['Lieu-dit'].unique()).intersection(set(dfNoDist['Lieu-dit'].unique())))

In [ ]:
# Au total, les sites / lieux-dits perdus sont ceux qui ont eut 0 donnée classée en distance
# lors d'au moins 1 inventaire (dfNoDist), et qui n'ont aucune autre donnée dans aucun autre inventaire.
assert set(dfStoc['Lieu-dit'].unique()) - set(dfStocIndiv['Lieu-dit'].unique()) \
       == set(dfNoDist['Lieu-dit'].unique()) \
          - set(dfStocIndiv['Lieu-dit'].unique()).intersection(set(dfNoDist['Lieu-dit'].unique()))

In [ ]:
len(dfStocIndiv.groupby(['Lieu-dit', 'Date']).first())

In [ ]:
len(dfStocIndiv.groupby(['Lieu-dit', 'Observateur', 'Date']).first())

### 6. Inventaires (points, observateurs, dates => effort d'inventaire par point)

In [ ]:
dfInventaires = \
  dfStocIndiv[['Lieu-dit', 'Observateur', 'Date', 'Nom latin']] \
    .groupby(['Lieu-dit', 'Observateur', 'Date']).first() \
    .reset_index().set_index('Lieu-dit').drop('Nom latin', axis=1)
dfInventaires.head()

In [ ]:
dfInventaires['Effort'] = dfInventaires.groupby('Lieu-dit').apply(lambda dfChk: len(dfChk)).astype(int)

In [ ]:
dfInventaires = dfInventaires.reset_index().set_index(['Lieu-dit', 'Observateur', 'Date'])
dfInventaires.head(10)

In [ ]:
# Nbre total d'inventaires
len(dfInventaires)

In [ ]:
len(dfInventaires.reset_index()['Lieu-dit'].unique())

In [ ]:
len(dfInventaires[dfInventaires['Effort'].isnull()])

In [ ]:
# Vérification que le fait de tenir compte des observateurs ne change rien à l'inventaire ... des inventaires
assert len(dfStocIndiv[['Lieu-dit', 'Date', 'Observateur']] \
            .groupby(['Lieu-dit', 'Date']).first() \
            .reset_index().set_index('Lieu-dit').drop('Observateur', axis=1)) \
       == len(dfInventaires)

In [ ]:
# Nbres d'inventaires par classe d'effort
dfInventaires.reset_index().groupby('Effort').count() \
             .drop(['Observateur', 'Date'], axis=1).rename(columns={'Lieu-dit' : 'Nb inventaires'})

In [ ]:
# Ajout de l'effort d'inventaire à chaque donnée individualisée
dfStocIndiv.set_index(['Lieu-dit', 'Observateur', 'Date'], inplace=True)
dfStocIndiv = dfStocIndiv.join(dfInventaires)
dfStocIndiv.reset_index(inplace=True)
dfStocIndiv.head()

### 7. Ajout des données d'absence

In [ ]:
# Ensemble complet des espèces d'intérêt (= contactées, puisqu'elles ont probablement toutes été contactées)
# NB: Cette méthode d'obtention est fiable, même si elle n'en a pas l'air \
# (espèces sans aucune donnée chaque année ? pas crédible)
stEspeces = set(dfStocIndiv['Nom latin'].unique())
print(len(stEspeces), 'espèces contactées :', ', '.join(sorted(stEspeces)))

In [ ]:
# Espèces contactées pour chaque lieu-dit (tous inventaires confondus)
dfEspecesContactees = dfStocIndiv.groupby(['Lieu-dit', 'Nom latin']).first().reset_index()[['Lieu-dit', 'Nom latin']]
dfEspecesContactees.head()

In [ ]:
# Espèces non contactées pour chaque lieu-dit (tous inventaires confondus)
# (=> un set d'espèces pour chaque site)
dfEspecesManquees = \
  dfEspecesContactees.groupby(['Lieu-dit']).agg(lambda sEsp: stEspeces - set(sEsp.unique()))

# Ajout des infos inventaire (effort)
dfEspecesManquees = dfEspecesManquees.join(dfInventaires.groupby('Lieu-dit').first())

In [ ]:
dfEspecesManquees.head()

In [ ]:
# Génération des données d'absence
dfStocAbsc = \
  pd.DataFrame(data=[{'Lieu-dit' : point, 'Observateur' : 'X', 'Date' : '',
                      'Nom latin' : espece, 'Distance' : np.nan, 'Effort' : sEspece.Effort } \
                     for point, sEspece in dfEspecesManquees.iterrows() \
                     for espece in sEspece['Nom latin']])
dfStocAbsc = dfStocAbsc.reindex(columns=dfStocIndiv.columns)

In [ ]:
# Vérification : Chaque espèce doit avoir au moins 1 ligne pour chaque site/lieu-dit inventorié au moins 1 fois
assert len(dfStocAbsc) + len(dfStocIndiv.groupby(['Lieu-dit', 'Nom latin']).first()) \
       - len(stEspeces) * len(dfInventaires.groupby('Lieu-dit').first()) == 0

In [ ]:
# Tablea final : Données de contact individualisé et d'absence.
dfStocTotal = dfStocIndiv.append(dfStocAbsc).sort_values(by=['Lieu-dit', 'Observateur', 'Date', 'Nom latin']) \
                    .reset_index().drop('index', axis=1)

In [ ]:
# Vérification rudimentaire en nombre d'espèces : on doit avoir 1 ligne par espèce et par point !
assert dfStocTotal.groupby(['Lieu-dit']) \
       .apply(lambda dfChk: len(dfChk['Nom latin'].unique()) == len(stEspeces)).all(), \
       "Attention : Il manque des données d'absence."

In [ ]:
len(dfStocTotal), len(dfStocIndiv), len(dfStocAbsc)

### 8. Export Excel (tout sur 1 feuille)

In [ ]:
xlsWriter = pd.ExcelWriter('STOC-EPS-{}-PourDistance.xlsx' \
                           .format(strAnnees), datetime_format='DD/MM/YYYY')
dfStocTotal.to_excel(xlsWriter, 'ToutesEspeces', index=False, float_format='%.1f')
xlsWriter.save()

### 9. Export CSV pour distance, pour une liste d'espèce, avec génération d'un log (stats)

In [ ]:
sorted(dfStoc['Nom latin'].unique())

In [ ]:
# Nom latin exact (Cf. colonne 'Nom latin' dans fichier Biolovision)
especes = ['Turdus merula', 'Fringilla coelebs', 'Sylvia atricapilla', 'Parus major', 'Phylloscopus collybita']

In [ ]:
dfInventaires2 = dfInventaires.reset_index()[['Lieu-dit', 'Effort']].groupby('Lieu-dit').first()
strLog = 'STOC EPS {}\n\n'.format('+'.join([str(annee) for annee in annees]))
strLog += '* Nbre de site / lieux-dits inventoriés : {}\n' .format(len(dfInventaires2))
strLog += '* Effort total (au sens de Distance) : {}\n'.format(dfInventaires2.Effort.sum())
strLog += '* Nombre des sites/lieux-dits par valeur d\'effort :\n'
strEffortTable = dfInventaires2.reset_index().groupby('Effort').count() \
                 .rename(columns={'Lieu-dit' : 'Nb points'}).to_string()
strLog += '\n'.join([' '*4 + s for s in strEffortTable.split('\n')]) + '\n'

for espece in especes:
    
    print(espece, '... ', end='')
    
    strLog += '* {}\n'.format(espece)
    dfCible = dfStocTotal[dfStocTotal['Nom latin'] == espece]

    if len(dfCible) == 0:
        print('Erreur : Aucune donnée !')
        strLog += '  Aucune donnée.'
        continue

    dfCible = dfCible[['Lieu-dit', 'Effort', 'Distance']]
    dfCible['Region'] = 'Auvergne'
    dfCible['Surface'] = 26000*100 # ha
    # Workaroung df.to_csv(float_format='%.1f') not working
    # when not only floats in the columns ...
    dfCible.Distance = \
      dfCible.Distance.apply(lambda x: '' if np.isnan(x) else '{:.1f}'.format(x).replace('.', ','))
    dfCible = \
      dfCible.reindex(columns=['Region', 'Surface', 'Lieu-dit', 'Effort', 'Distance'])
    dfCible.sort_values(by=['Lieu-dit'], inplace=True)

    assert set(dfCible['Lieu-dit'].unique()) == set(dfInventaires.reset_index()['Lieu-dit'].unique())

    strLog += '  - Nbre de données individuelles : total {}, d\'absence {}\n' \
              .format(len(dfCible), len(dfCible[dfCible.Distance == '']))

    # Export final.
    tgtFileName = ''.join([word.capitalize() for word in espece.split(' ')]) + '-' + strAnnees + '-dist.txt'
    dfCible.to_csv(tgtFileName, index=False, sep='\t', encoding='utf-8',
                   header=['Region*Label', 'Region*Area', 'Point transect*Label',
                           'Point transect*Survey effort', 'Observation*Radial distance'])

    print(tgtFileName)

print()
print(strLog)
print(strLog, file=open('STOC-EPS-' + strAnnees + '-dist.log', 'w'))

In [ ]:
raise Exception('Rien de grave, c\'est juste terminé :-)')

## Fichier Pierre STOC fin 2015
    (pas cohérent avec envoi Romain mi 2017, mais avec les habitats ...)

In [ ]:
dfToutBrutPT = \
  pd.read_csv('Pierre2015/STOC-Auvergne-2015-utf8.csv', parse_dates=['Date'], infer_datetime_format=True, 
              low_memory=False, index_col=0, sep='\t', decimal='.', encoding='utf-8', skiprows=1)

In [ ]:
len(dfToutBrutPT)

In [ ]:
dfToutBrutPT.columns

In [ ]:
dfToutBrutPT.head()

In [ ]:
len(dfToutBrutPT['Nom latin'].unique())

In [ ]:
dfTurMerPT = dfToutBrutPT[dfToutBrutPT['Nom latin'] == 'Turdus merula']
assert len(dfTurMerPT) == 532, 'Nb données TurMer non concordant avec filtrage LibreOfficeCalc'

In [ ]:
# Nb de points STOC
assert len(dfTurMerPT['Lieu-dit'].unique()) == 338, 'Nb points avec TurMer non concordant avec filtrage LibreOfficeCalc'
assert len(dfToutBrutPT['Lieu-dit'].unique()) == 381, 'Nb total points inventorié non concordant avec filtrage LibreOfficeCalc'

In [ ]:
# Nb d'inventaires
assert len(dfTurMerPT.groupby(['Date', 'Lieu-dit']).first()) in (531, 532),  'Nb inventaires avec TurMer non concordant avec filtrage LibreOfficeCalc'
assert len(dfToutBrutPT.groupby(['Date', 'Lieu-dit']).first()) == 708,  'Nb total inventaire non concordant avec filtrage LibreOfficeCalc'

In [ ]:
dfTurMerPT['Nombre'].sum()

In [ ]:
dfToutBrutPT['Habitat principal 1'].unique()